Advanced Tricks
===

So you've tried making normal waveforms and now you need to spice up your life by making some way more weird waveforms letting the detector be whatever you want it to be? 
You have come to the right place!

By default fax uses some configuration file which is a huge pain to modify. So we made fax such that if you add a parameter in the instruction which corresponds to a parameter in the config it will overwrite what the value was in the config and let you deside what it should be!

This example shows how to modify the electron lifetime and the anode voltage

In [1]:
import numpy as np
import strax
import straxen
import wfsim

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from multihist import Histdd, Hist1d
from scipy import stats

In [2]:
st = strax.Context(
    config=dict(
        detector='XENON1T',
        fax_config='https://raw.githubusercontent.com/XENONnT/'
                   'strax_auxiliary_files/master/sim_files/fax_config_1t.json',
        fax_config_override={'field_distortion_on':True, 's2_luminescence_model':'simple'},
        **straxen.contexts.xnt_common_config),
    **straxen.contexts.common_opts)
st.register(wfsim.RawRecordsFromFax1T)

wfsim.strax_interface.RawRecordsFromFax1T

In [3]:
# Just some id from post-SR1, so the corrections work
run_id = '000001'

In [4]:
strax.Mailbox.DEFAULT_TIMEOUT=10000

In [5]:
dtype = wfsim.strax_interface.instruction_dtype

for new_dtype in [('electron_lifetime_liquid', np.int32),
                  ('anode_voltage', np.int32)]:
    if new_dtype not in dtype:
        dtype.append(new_dtype)

def rand_instructions(c):
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    instructions = np.zeros(2 * n, dtype=dtype)
    uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
    instructions['time'] = np.repeat(uniform_times, 2) * int(1e9)
    instructions['event_number'] = np.digitize(instructions['time'], 
         1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    instructions['type'] = np.tile([1, 2], n)
    instructions['recoil'] = ['er' for i in range(n * 2)]

    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    instructions['x'] = np.repeat(r * np.cos(t), 2)
    instructions['y'] = np.repeat(r * np.sin(t), 2)
    instructions['z'] = np.repeat(np.random.uniform(-100, 0, n), 2)

    nphotons = np.random.uniform(2000, 2050, n)
    nelectrons = 10 ** (np.random.uniform(1, 4, n))
    instructions['amp'] = np.vstack([nphotons, nelectrons]).T.flatten().astype(int)
    instructions['electron_lifetime_liquid'] = np.repeat(600e10,len(instructions))
    instructions['anode_voltage'] = np.repeat(1e10,len(instructions))
    return instructions

wfsim.strax_interface.rand_instructions = rand_instructions
wfsim.strax_interface.instruction_dtype = dtype

In [6]:
st.set_config(dict(fax_file=None))
st.set_config(dict(nchunk=1, event_rate=1, chunk_size=100))

In [10]:
# Remove any previously simulated data, if such exists
# !rm -r strax_data

records = st.get_array(run_id,'raw_records', progress_bar=False)
peaks = st.get_array(run_id, ['peak_basics'], progress_bar=False)
data = st.get_df(run_id, 'event_info', progress_bar=False)
truth = st.get_df(run_id, 'truth', progress_bar=False)

In [11]:
truth.head()

,event_number,type,time,x,y,z,amp,recoil,electron_lifetime_liquid,anode_voltage,...,n_photon_bottom,t_first_photon,t_last_photon,t_mean_photon,t_sigma_photon,t_first_electron,t_last_electron,t_mean_electron,t_sigma_electron,endtime
0,0,1,500000026,-27.127699,-36.640770,-12.655296,2009,er,-2147483648,-2147483648,...,143.0,5.000000e+08,5.000004e+08,5.000001e+08,48.421194,NaN,NaN,NaN,NaN,500000353
1,0,2,500097027,-27.127699,-36.640770,-12.655296,39,er,-2147483648,-2147483648,...,344.0,5.000970e+08,5.000997e+08,5.000984e+08,490.645247,5.000970e+08,5.000993e+08,5.000982e+08,479.974221,500099672
2,0,4,500768320,20.171877,4.092766,-89.123016,1,er,-2147483648,-2147483648,...,10.0,5.007683e+08,5.007690e+08,5.007685e+08,167.279008,5.007683e+08,5.007683e+08,5.007683e+08,0.000000,500769026
3,0,1,1500000019,-19.476303,10.349952,-35.648590,2010,er,-2147483648,-2147483648,...,192.0,1.500000e+09,1.500000e+09,1.500000e+09,41.286365,NaN,NaN,NaN,NaN,1500000227
4,0,2,1500265504,-19.476303,10.349952,-35.648590,2817,er,-2147483648,-2147483648,...,26229.0,1.500266e+09,1.500273e+09,1.500269e+09,961.919333,1.500265e+09,1.500272e+09,1.500269e+09,955.008653,1500272834
